In [1]:
import argparse
import asyncio
import json
import logging
import os
import sys

import nest_asyncio
import sqlalchemy.exc
from dotenv import load_dotenv
from sqlalchemy import select, text, delete, func
from sqlalchemy.ext.asyncio import async_sessionmaker

from fastapi_app.postgres_engine import create_postgres_engine_from_args, create_postgres_engine_from_env
from fastapi_app.postgres_models import Item

logger = logging.getLogger("ragapp")

nest_asyncio.apply()

async def seed_data(engine):
    async with engine.begin() as conn:
        # Check if the Item table exists
        result = await conn.execute(
            text(
                "SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_schema = 'public' AND table_name = 'items')"
            )
        )
        if not result.scalar():
            logger.error("Items table does not exist. Please run the database setup script first.")
            return

        # Delete all entries in the Item table
        await conn.execute(delete(Item))
        await conn.commit()

        # Check if the delete operation was successful
        count_result = await conn.execute(select([func.count()]).select_from(Item))
        count = count_result.scalar()
        if count == 0:
            logger.info("Successfully deleted all entries in the Items table.")
        else:
            logger.warning(f"Failed to delete all entries, {count} entries remain.")

    async with async_sessionmaker(engine, expire_on_commit=False)() as session:
        current_dir = os.path.dirname(os.path.realpath(__file__))
        with open(os.path.join(current_dir, "seed_lse_data.json")) as f:
            catalog_items = json.load(f)
            for catalog_item in catalog_items:
                item_id = int(catalog_item["Id"])
                item = await session.execute(select(Item).filter(Item.id == item_id))
                if item.scalars().first():
                    continue
                item = Item(
                    id=item_id,
                    name=catalog_item["Name"],
                    description=catalog_item["Description"],
                    embedding=catalog_item["Embedding"],
                    link=catalog_item["Link"]
                )
                session.add(item)

            try:
                await session.commit()
            except sqlalchemy.exc.IntegrityError:
                pass

    logger.info("Items table seeded successfully.")

def parse_args():
    parser = argparse.ArgumentParser(description="Create database schema")
    parser.add_argument("--host", type=str, help="Postgres host", default=os.getenv("POSTGRES_HOST"))
    parser.add_argument("--username", type=str, help="Postgres username", default=os.getenv("POSTGRES_USER"))
    parser.add_argument("--password", type=str, help="Postgres password", default=os.getenv("POSTGRES_PASSWORD"))
    parser.add_argument("--database", type=str, help="Postgres database", default=os.getenv("POSTGRES_DB"))
    parser.add_argument("--sslmode", type=str, help="Postgres sslmode", default=os.getenv("POSTGRES_SSLMODE"))

    if len(sys.argv) == 1:  # If no arguments, assume running in an environment like Jupyter
        return parser.parse_args(args=[])
    else:
        return parser.parse_args()

async def main():
    args = parse_args()
    if args.host:
        engine = await create_postgres_engine_from_args(args)
    else:
        engine = await create_postgres_engine_from_env()

    await seed_data(engine)
    await engine.dispose()

if __name__ == "__main__":
    logging.basicConfig(level=logging.WARNING)
    logger.setLevel(logging.INFO)
    load_dotenv(override=True)
    asyncio.run(main())


usage: ipykernel_launcher.py [-h] [--host HOST] [--username USERNAME]
                             [--password PASSWORD] [--database DATABASE]
                             [--sslmode SSLMODE]
ipykernel_launcher.py: error: unrecognized arguments: --f=/Users/akshsabherwal/Library/Jupyter/runtime/kernel-v2-68291Wvz9RVWuPTE.json


SystemExit: 2

/Users/akshsabherwal/Desktop/chat-lse/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
